# Label encode TensorFlow


In [1]:
import os
import sys
import zipfile
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

sys.path.insert(0,os.path.join(os.getcwd(), os.pardir,  'src', 'data'))
import stack_data

sys.path.insert(0,os.path.join(os.getcwd(), os.pardir,  'src', 'features'))
import encoded

SHOW_DISPLAY = True

In [2]:
 # Fetch the data
raw_data = stack_data.get_data()

print(raw_data.shape)
if SHOW_DISPLAY:
    display(raw_data.head())

(51392, 8)


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,DeveloperType
0,Student,"Yes, both",United States,No,Secondary school,NaN,2 to 3 years,NaN
1,Student,"Yes, both",United Kingdom,"Yes, full-time",Some college/university study without earning ...,Computer science or software engineering,9 to 10 years,NaN
2,Professional developer,"Yes, both",United Kingdom,No,Bachelor's degree,Computer science or software engineering,20 or more years,Other
3,Professional non-developer who sometimes write...,"Yes, both",United States,No,Doctoral degree,A non-computer-focused engineering discipline,14 to 15 years,NaN
4,Professional developer,"Yes, I program as a hobby",Switzerland,No,Master's degree,Computer science or software engineering,20 or more years,Mobile developer; Graphics programming; Deskto...


In [3]:
# Label encode categorical data 
raw_data = encoded.get_encoded(raw_data, stack_data.LABEL_NAME)
print(raw_data.shape)
if SHOW_DISPLAY:
    display(raw_data.head())

(16747, 8)


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,DeveloperType
0,0,3,143,1,0,8,12,10
1,0,3,106,1,4,8,1,13
2,0,0,23,1,0,8,4,13
3,0,3,45,1,8,8,6,13
4,0,2,143,1,6,6,12,5


In [4]:
# Split to train and test data.
# TODO: Consider cross validation
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6
train, test = train_test_split(raw_data, train_size = 0.8, test_size = 0.2)
if SHOW_DISPLAY:
    display(train.head())
    display(test.head())

,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,DeveloperType
5535,0,0,125,1,6,11,19,12
3673,0,2,74,2,7,16,16,13
3076,0,2,144,1,7,16,13,13
417,0,3,144,1,0,6,7,2
15174,0,3,56,2,0,8,15,13


,Professional,ProgramHobby,Country,University,FormalEducation,MajorUndergrad,YearsProgram,DeveloperType
3697,0,2,143,1,4,3,15,2
2570,0,3,56,1,4,8,15,13
8502,0,2,57,3,7,16,0,13
3972,0,2,74,2,0,8,15,2
13838,0,0,144,1,0,8,12,13


In [5]:
# 1. Assign the DataFrame's labels (the right-most column) to train_label.
# 2. Delete (pop) the labels from the DataFrame.
# 3. Assign the remainder of the DataFrame to train_features
X_train, Y_train = train, train.pop(stack_data.LABEL_NAME)
X_test, Y_test = test, test.pop(stack_data.LABEL_NAME)

In [6]:
# Create feature columns for all features.
my_feature_columns = []
for key in X_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
if SHOW_DISPLAY:
    display(my_feature_columns)

[_NumericColumn(key='Professional', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='ProgramHobby', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Country', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='University', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='FormalEducation', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='MajorUndergrad', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='YearsProgram', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [7]:
# Instantiate DNNClassifier
classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        hidden_units=[10, 10],
        n_classes=14)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\vagrant\\AppData\\Local\\Temp\\tmpwyhgxdec', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000272884B8CC0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# Train model
BATCH_SIZE = 100
TRAIN_STEPS = 1000

def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

classifier.train(
    input_fn=lambda:train_input_fn(X_train, Y_train, BATCH_SIZE),
    steps=TRAIN_STEPS)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\vagrant\AppData\Local\Temp\tmpwyhgxdec\model.ckpt.
INFO:tensorflow:loss = 5379.879, step = 1
INFO:tensorflow:global_step/sec: 825.322
INFO:tensorflow:loss = 151.75328, step = 101 (0.121 sec)
INFO:tensorflow:global_step/sec: 972.529
INFO:tensorflow:loss = 168.43884, step = 201 (0.103 sec)
INFO:tensorflow:global_step/sec: 878.425
INFO:tensorflow:loss = 127.273125, step = 301 (0.114 sec)
INFO:tensorflow:global_step/sec: 1146.5
INFO:tensorflow:loss = 149.22697, step = 401 (0.106 sec)
INFO:tensorflow:global_step/sec: 759.948
INFO:tensorflow:loss = 121.577095, step = 501 (0.113 sec)
INFO:tensorflow:global_step/sec: 887.77
INFO:tensorflow:loss = 118.29767, step = 601 (0.116 sec)
INFO:tensorflow:global_step/sec: 1158.77
INFO:tensorflow:loss = 146.26758, step = 701 (0.083 sec)
INFO:tensorflow:global_step/sec: 1174.6
INFO:tensorflow:loss = 160.06677, step = 801 (0.085 sec)
INFO:tensorflow:global_st

In [9]:
 # Evaluate the model.
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

    
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(X_test, Y_test,
                                            BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Starting evaluation at 2018-02-20-18:33:15
INFO:tensorflow:Restoring parameters from C:\Users\vagrant\AppData\Local\Temp\tmpwyhgxdec\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-20-18:33:15
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.64567167, average_loss = 1.3496914, global_step = 1000, loss = 132.9843

Test set accuracy: 0.646

